In [16]:
from fastapi import FastAPI
from fastapi.params import Body
from fastapi import Response
from pydantic import BaseModel
from typing import Optional
from fastapi import status
from fastapi import HTTPException
import psycopg2.extras

In [15]:
# macros, initials
cuisines = ["Turkish", "Italian", "French", "Lebanese", "Japanese"]

In [17]:
# database connection and initialization
try:
    connection = psycopg2.connect(host='0.0.0.0', database='postgres', 
                user='postgres', password='fastapi24', cursor_factory=psycopg2.extras.RealDictCursor)
    cursor = connection.cursor()
    #connection.setAutoCommit(True)
    print("[INFO] Database connected successfully!")
except Exception as databaseException:
    print("[INFO] Connecting to database failed! ", databaseException)

[INFO] Database connected successfully!


In [ ]:
def create_cuisines_table():
    try:
        sql_query = (""" CREATE TABLE IF NOT EXISTS cuisines (
                            cuisine_id SERIAL PRIMARY KEY,
                            cuisine_name VARCHAR(255) NOT NULL
                        )""")
        cursor.execute(sql_query)
    except Exception as e:
        print("[INFO] ", e)

In [ ]:
def insert_cuisines_table():
    sql_query = '''INSERT INTO cuisines (cuisine_name) VALUES (%s)'''
    for cuisine in cuisines:
        cursor.execute(sql_query, (cuisine,))

In [ ]:
create_cuisines_table()
connection.commit()

In [ ]:
insert_cuisines_table()
connection.commit()

In [10]:
from PIL import Image
import requests
from io import BytesIO
import openpyxl

url = "https://spoonacular.com/recipeImages/648606-312x231.jpg"
response = requests.get(url)
img = BytesIO(response.content)
db_image = img.getbuffer().tobytes()


In [20]:
def create_foods_table():
    try:
        sql_query = (""" CREATE TABLE IF NOT EXISTS foods (
                            food_id SERIAL PRIMARY KEY,
                            food_name VARCHAR(255) NOT NULL,
                            food_image BYTEA,
                            food_price INTEGER NOT NULL,
                            cuisine_id INTEGER REFERENCES cuisines (cuisine_id)
                        )""")
        cursor.execute(sql_query)
    except Exception as e:
        print("[INFO] ", e)


In [21]:
create_foods_table()
connection.commit()